In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('/Users/dhruvpatel/Desktop/projects/DealPredection/data/vehicles.csv')

print(df.head())

           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       NaN  ...  NaN   NaN

# EDA

In [2]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

In [3]:
## drop the columns that are not needed 
df_cleaned=df
df_cleaned=df_cleaned.drop(['url','image_url','county','VIN','size','condition'],axis=1) 

In [4]:
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
drive           130567
type             92858
paint_color     130203
description         70
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

In [5]:
# dropping rows due to NAs 

df_cleaned =df_cleaned.dropna(subset=['year','description','fuel','odometer','lat','long','transmission','model'])

In [6]:
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer     15836
model                0
cylinders       168709
fuel                 0
odometer             0
title_status      6778
transmission         0
drive           124295
type             88728
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [7]:
len(df)-len(df_cleaned)

20827

# Filling missing values 

In [8]:
# filling missing values in title status with 'missing' 

df_cleaned['title_status'] = df_cleaned['title_status'].fillna('missing')
df_cleaned.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer     15836
model                0
cylinders       168709
fuel                 0
odometer             0
title_status         0
transmission         0
drive           124295
type             88728
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [9]:
df_cleaned['transmission'] = df_cleaned['transmission'].fillna('other')

In [10]:
# filling null values from the model. 
from model_extract import process_car_dataset

featured_eng=process_car_dataset(df_cleaned)


Starting data extraction and cleaning...
Data extraction and cleaning completed!


In [11]:
# increase = featured_eng['drive'].value_counts() - df_cleaned['drive'].value_counts()
# print(increase)

In [12]:
# null values reduced 
featured_eng.isnull().sum() - df_cleaned.isnull().sum()

id                  0
region              0
region_url          0
price               0
year                0
manufacturer   -12111
model               0
cylinders      -30367
fuel                0
odometer            0
title_status        0
transmission        0
drive          -71030
type           -40512
paint_color         0
description         0
state               0
lat                 0
long                0
posting_date        0
dtype: int64

In [13]:
df_cleaned['cylinders'].unique()

array(['8 cylinders', '6 cylinders', nan, '4 cylinders', '5 cylinders',
       'other', '3 cylinders', '10 cylinders', '12 cylinders'],
      dtype=object)

In [14]:
featured_eng.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer      3725
model                0
cylinders       138342
fuel                 0
odometer             0
title_status         0
transmission         0
drive            53265
type             48216
paint_color     123699
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [15]:
# handling the maufacturer column  

# prinitng the values where manufecturer is null 

manufecturer_null = featured_eng[featured_eng['manufacturer'].isna()]

# printing the values where manufecturer is null 


manufecturer_null['model'].value_counts()





model
Janesville                          49
PROGRAMS as little as $500 DOWN!    48
ALL                                 35
chryler town and country            28
ALL MAKES                           28
                                    ..
Chevelle  s10                        1
HHR 1LT                              1
1992 corvette  convertible           1
94 mustang convertible gt 5.0        1
Paige Glenbrook Touring              1
Name: count, Length: 2366, dtype: int64

In [16]:
# dropping the rows where manufacturer is null 

featured_eng = featured_eng[featured_eng['manufacturer'].notna()]



# drive column cleaning 

In [17]:
featured_eng['drive'].value_counts()

drive
4wd                  182408
fwd                  104328
rwd                   59363
allwheeldrive          2883
frontwheeldrive        1285
rearwheeldrive          637
front wheel drive         1
Name: count, dtype: int64

In [18]:
from drive_cleaning import process_drive_column
featured_eng=process_drive_column(featured_eng, 'drive')


Starting drive column cleaning...
Drive column cleaning completed!
Updated distribution:
drive
4wd    185291
fwd    105614
rwd     60000
Name: count, dtype: int64
DRIVE COLUMN CLEANING REPORT

BEFORE CLEANING:
------------------------------
drive
4wd                  182408
fwd                  104328
rwd                   59363
allwheeldrive          2883
frontwheeldrive        1285
rearwheeldrive          637
front wheel drive         1
Name: count, dtype: int64

AFTER CLEANING:
------------------------------
drive
4wd    185291
fwd    105614
rwd     60000
Name: count, dtype: int64

CHANGES:
------------------------------
allwheeldrive: 2883 → 0 (-2883)
4wd: 182408 → 185291 (+2883)
front wheel drive: 1 → 0 (-1)
rwd: 59363 → 60000 (+637)
frontwheeldrive: 1285 → 0 (-1285)
fwd: 104328 → 105614 (+1286)
rearwheeldrive: 637 → 0 (-637)

VALIDATION:
------------------------------
Total rows: 402328
Null values: 51423
Unexpected values: []
Column is clean: True

Drive column cleaning complete

In [19]:
# filling null values from research :
from drive_cleaning import fill_missing_drive_from_reference


featured_eng = fill_missing_drive_from_reference(featured_eng,reference_file='/Users/dhruvpatel/Desktop/projects/DealPredection/data/models_with_drive.csv')






 

Loading reference data from: /Users/dhruvpatel/Desktop/projects/DealPredection/data/models_with_drive.csv
Loaded 83 model-drive mappings from reference file
Found 51423 rows with missing drive values
✓ Filled drive for model 'focus' → 'fwd'
✓ Filled drive for model 'optima' → 'fwd'
✓ Filled drive for model 'versa' → 'fwd'
✓ Filled drive for model 'monte carlo' → 'fwd'
✓ Filled drive for model 'accord ex-l' → 'fwd'
✓ Filled drive for model 'jetta' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model 'mustang' → 'rwd'
✓ Filled drive for model 'impala lt' → 'fwd'
✓ Filled drive for model 'avalon' → 'fwd'
✓ Filled drive for model 'cooper s' → 'fwd'
✓ Filled drive for model 'accord' → 'fwd'
✓ Filled drive for model 'fusion se' → 'fwd'
✓ Filled drive for model 'lesabre' → 'fwd'
✓ Filled drive for model 'maxima' → 'fwd'
✓ Filled drive for model 'maxima' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model '200' → 'fwd'
✓ Filled drive for model 'sonata' → 

In [20]:
featured_eng['drive'].isna().sum()

np.int64(38617)

In [21]:
featured_eng['drive'].value_counts()

drive
4wd    185291
fwd    116757
rwd     61663
Name: count, dtype: int64

In [22]:
featured_eng.isnull().sum()

id                   0
region               0
region_url           0
price                0
year                 0
manufacturer         0
model                0
cylinders       136673
fuel                 0
odometer             0
title_status         0
transmission         0
drive            38617
type             46908
paint_color     122057
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

# Cleaning Model 

Clean in stages:
* Stage 1: Remove obvious junk (numbers, too short, too long)
* Stage 2: Extract core model from complex strings
* Stage 3: Standardize spelling and format
* Stage 4: Apply manufacturer-specific rules

In [23]:
# Stage 1 : Remove obvious junk (numbers, too short, too long)

from model_cleaning import remove_numerical_models

featured_eng=remove_numerical_models(featured_eng)

featured_eng['model'].value_counts().to_csv('model_counts.csv')






/Users/dhruvpatel/miniconda3/envs/Deal_Predection/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Debug: DataFrame shape after cleaning: (388894, 20)
Step: remove_numerical_models
Rows removed due to numerical only: 11783
Rows removed due to length > 40: 1651
Total rows removed: 13434


In [24]:
# stage 2 : 

In [25]:
featured_eng['manufacturer'].unique()

array(['gmc', 'chevrolet', 'toyota', 'ford', 'jeep', 'nissan', 'ram',
       'mazda', 'cadillac', 'honda', 'dodge', 'lexus', 'jaguar', 'buick',
       'chrysler', 'volvo', 'audi', 'infiniti', 'lincoln', 'alfa-romeo',
       'subaru', 'scion', 'acura', 'hyundai', 'mercedes-benz', 'bmw',
       'mitsubishi', 'smart', 'volkswagen', 'porsche', 'kia', 'sterling',
       'freightliner', 'maserati', 'international', 'saab', 'suzuki',
       'ferrari', 'mini', 'pontiac', 'plymouth', 'oldsmobile', 'fiat',
       'rover', 'genesis', 'hummer', 'isuzu', 'hino', 'kenworth', 'tesla',
       'saturn', 'am general', 'mercury', 'willys', 'bentley', 'nash',
       'mg', 'triumph', 'peterbilt', 'geo', 'harley-davidson', 'amc',
       'kaiser', 'lamborghini', 'edsel', 'austin', 'rolls-royce',
       'datsun', 'studebaker', 'mclaren', 'polaris', 'mack', 'hudson',
       'desoto', 'lotus', 'aston-martin', 'crosley', 'packard',
       'western star', 'land rover', 'morris', 'eagle', 'daewoo',
       'daihats

In [26]:
# featured_eng.to_csv('cleaned_data.csv')


In [27]:
# import pandas as pd 
# featured_eng = pd.read_csv('cleaned_data.csv')

In [28]:
featured_eng['model'].value_counts()


model
f-150                         7528
silverado 1500                4772
camry                         3054
silverado                     2948
accord                        2891
                              ... 
IH D1300                         1
2 series 228i xdrive coupe       1
express 2500hd                   1
liborty                          1
96 Suburban                      1
Name: count, Length: 25578, dtype: int64

In [29]:
df_cleaned['cylinders'].unique()

array(['8 cylinders', '6 cylinders', nan, '4 cylinders', '5 cylinders',
       'other', '3 cylinders', '10 cylinders', '12 cylinders'],
      dtype=object)

In [30]:
from model_cleaning import clean_models_with_list
 
featured_eng_new = clean_models_with_list(featured_eng)

Starting model and manufacturer cleaning...
Extracting valid models...


Cleaning Models: 100%|██████████| 388894/388894 [00:02<00:00, 136271.33it/s]


Found 231596 models to update.
Step: clean_models_with_list
Total rows modified: 153935


In [31]:
df_cleaned['cylinders'].unique()

array(['8 cylinders', '6 cylinders', nan, '4 cylinders', '5 cylinders',
       'other', '3 cylinders', '10 cylinders', '12 cylinders'],
      dtype=object)

In [32]:
featured_eng_new['model'].value_counts()

model
1500                   17993
super                   7580
f-150                   7528
civic                   5610
tacoma                  5430
                       ...  
2012 Mustang               1
duramax crew               1
AVALANCE NORTH FACE        1
VMI-CHRYSLER ♿             1
gand wagoneer              1
Name: count, Length: 10722, dtype: int64

In [ ]:
featured_eng_new.to_csv('clean_v1.csv')

In [5]:
import pandas as pd 
clean_df =pd.read_csv('clean_v1.csv')
clean_df.head()

,Unnamed: 0,id,region,region_url,price,year,manufacturer,model,cylinders,fuel,...,title_status,transmission,drive,type,paint_color,description,state,lat,long,posting_date
0,27,7316814884,auburn,https://auburn.craigslist.org,33590,2014.0,ram,1500,8 cylinders,gas,...,clean,other,4wd,pickup,white,Carvana is the safer way to buy a car During t...,al,32.590,-85.4800,2021-05-04T12:31:18-0500
1,28,7316814758,auburn,https://auburn.craigslist.org,22590,2010.0,ram,1500,8 cylinders,gas,...,clean,other,4wd,pickup,blue,Carvana is the safer way to buy a car During t...,al,32.590,-85.4800,2021-05-04T12:31:08-0500
2,29,7316814989,auburn,https://auburn.craigslist.org,39590,2020.0,ram,1500,8 cylinders,gas,...,clean,other,4wd,pickup,red,Carvana is the safer way to buy a car During t...,al,32.590,-85.4800,2021-05-04T12:31:25-0500
3,30,7316743432,auburn,https://auburn.craigslist.org,30990,2017.0,toyota,tundra double cab sr,8 cylinders,gas,...,clean,other,4wd,pickup,red,Carvana is the safer way to buy a car During t...,al,32.590,-85.4800,2021-05-04T10:41:31-0500
4,31,7316356412,auburn,https://auburn.craigslist.org,15000,2013.0,ford,f-150 xlt,6 cylinders,gas,...,clean,automatic,rwd,truck,black,2013 F-150 XLT V6 4 Door. Good condition. Leve...,al,32.592,-85.5189,2021-05-03T14:02:03-0500


In [8]:
clean_df.isnull().sum()

Unnamed: 0           0
id                   0
region               0
region_url           0
price                0
year                 0
manufacturer         0
model                0
cylinders       132243
fuel                 0
odometer             0
title_status         0
transmission         0
drive            37445
type             45575
paint_color     117330
description          0
state                0
lat                  0
long                 0
posting_date         0
dtype: int64

In [13]:
clean_df['drive'].value_counts()
 

drive
4wd    176674
fwd    114968
rwd     59807
Name: count, dtype: int64

In [12]:
clean_df['drive'].isna().sum() 
 

np.int64(37445)

In [ ]:
# Assumption : As the dataset is batewnn 2018 - 2022 , reports says almost 40-50% of the dars were rear wheel drive 

In [14]:
clean_df['type'].value_counts()

type
sedan          86026
SUV            80463
truck          43415
pickup         40589
coupe          19572
other          19074
hatchback      16781
van            11252
wagon          11241
convertible     8368
mini-van        4917
bus              720
offroad          714
mini van         187
Name: count, dtype: int64

In [ ]:
# Method 1: Using crosstab with percentages
drive_type_breakdown = pd.crosstab(clean_df['type'], clean_df['drive'], normalize='index') * 100
print("Drive Type Distribution by Car Type (Percentages):")
print(drive_type_breakdown.round(2))

print("\n" + "="*60 + "\n")


# based on the data , lets fill the null values with condition.  
# Assumptions : 
# 1 ) if the vehicle type are SUV ,  offroad , pickup , truck, other or wagon then fill ->  drive as 4wd 
# 2) if vehicle type are hatchback, minivan, sedan, van then fill -> drive fwd  
# 3 if  vehicle  are bus,convertible,coupe then fill rwd  
# for other 


Drive Type Distribution by Car Type (Percentages):
drive          4wd    fwd    rwd
type                            
SUV          74.11  21.09   4.80
bus          10.71   7.35  81.93
convertible  10.16  17.99  71.85
coupe         9.59  23.04  67.36
hatchback    26.99  68.96   4.05
mini van     20.95  78.38   0.68
mini-van      5.84  90.47   3.69
offroad      95.02   1.61   3.37
other        60.27  21.08  18.65
pickup       82.28   3.91  13.81
sedan        23.23  61.58  15.19
truck        76.22   1.86  21.92
van          16.59  44.11  39.31
wagon        57.01  39.42   3.57


